# Import libraries

In [ ]:
import numpy as np

import cv2
from google.colab.patches import cv2_imshow

import time

import os

# Connection with Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Variables

In [ ]:
video_name = 'obsea'

In [ ]:
basic_path = '/content/drive/MyDrive/'
path_frames = basic_path + 'frames_saved/' + str(video_name) + '/'
path_video = basic_path + 'output_videos/'

# Functions

In [ ]:
# This function makes a video with the images given.
# Parameters: 
#              outvid   ->    Path where the output video will be saved
#              images   ->    List of images to use in the video
#              fps      ->    Frame per second
#              size     ->    Size of each frame
#              is_color ->    True or False
#              format   ->    Format of the output video
    
def make_video(outvid, images=None, fps=30, size=None, is_color=True, format="FMP4"):

    from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize

    fourcc = VideoWriter_fourcc(*format)
    vid = None

    for image in images:
        if not os.path.exists(image):
            raise FileNotFoundError(image)
        img = imread(image)
        if vid is None:
            if size is None:
                size = img.shape[1], img.shape[0]
            vid = VideoWriter(outvid, fourcc, float(fps), size, is_color)
        if size[0] != img.shape[1] and size[1] != img.shape[0]:
            img = resize(img, size)
        vid.write(img)

    vid.release()

    return vid

# Folder creation

In [ ]:
os.mkdir(path_frames)

# Movement detection

In [ ]:
camera = cv2.VideoCapture(basic_path + video_name + '.mp4')

In [ ]:
background = None
currentframe = 0

In [ ]:
counter = 0

while True and counter <100:

  counter = counter+1

  (grabbed, frame) = camera.read()

  if not grabbed:
    break

  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

  gray = cv2.GaussianBlur(gray, (21, 21), 0)

  if background is None:
    background = gray
    continue
  
  subtraction = cv2.absdiff(background, gray)

  threshold, outlinesimg = cv2.threshold(subtraction, 25, 255, cv2.THRESH_BINARY)

  threshold = cv2.dilate(threshold, None, iterations = 2)

  contourimg = threshold.copy()

  outlines, hierarchy = cv2.findContours(outlinesimg, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

  for c in outlines:
    if cv2.contourArea(c) < 500:
      continue

    (x, y, w, h) = cv2.boundingRect(c)

    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

  cv2_imshow(frame)
  cv2_imshow(threshold)
  cv2_imshow(subtraction)
  cv2_imshow(contourimg)

  name = path_frames + 'frame' + str(currentframe) + '_' + video_name + '.jpg'
  print ('Creating...' + name)

  cv2.imwrite(name, frame)

  currentframe = currentframe + 1

  key = cv2.waitKey(1) & 0xFF

  if key == ord('s'): 
    break

camera.release()
cv2.destroyAllWindows() 

# Creating video with movement detected

In [ ]:
# Directory of images to run detection on
images = os.listdir(path_frames)

In [ ]:
new_images = []

for image in images:
  new_images.append(path_frames + image)

In [ ]:
outvid = os.path.join(path_video, video_name + "_out.mp4")

In [ ]:
make_video(outvid, new_images, fps=30)